# Задание 1. A/B–тестирование

### Условие

Одной из основных задач аналитика в нашей команде является корректное проведение экспериментов. Для этого мы применяем метод A/B–тестирования. В ходе тестирования одной гипотезы целевой группе была предложена новая механика оплаты услуг на сайте, у контрольной группы оставалась базовая механика. В качестве задания Вам необходимо проанализировать итоги эксперимента и сделать вывод, стоит ли запускать новую механику оплаты на всех пользователей.

In [3]:
import pandas as pd

# библиотеки для чтения данных
import requests as r
import json

### Входные данные

В качестве входных данных имеется 4 csv-файла:

- groups.csv - файл с информацией о принадлежности пользователя к контрольной или экспериментальной группе (А – контроль, B – целевая группа) 
    
- groups_add.csv - дополнительный файл с пользователями, который вам прислали спустя 2 дня после передачи данных
    
- active_studs.csv - файл с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента. 
    
- checks.csv - файл с информацией об оплатах пользователей в дни проведения эксперимента. 


### Считывание данных

In [34]:
# Функция для считывания данных с Яндекс Диска

def read_csv_file(link):
    file = r.get(f'https://cloud-api.yandex.net/v1/disk/public/resources?public_key={link}').json()['file']
     # при помощи параметра engine='python' в параметре sep удалось учесть несколько разделителей
    df = pd.read_csv(file, sep=';|,', engine='python')
    return df

In [29]:
# Ссылки на данные

groups_link = 'https://disk.yandex.ru/d/UhyYx41rTt3clQ'
groups_add_link = 'https://disk.yandex.ru/d/5Kxrz02m3IBUwQ'
active_studs_link = 'https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ'
checks_link = 'https://disk.yandex.ru/d/pH1q-VqcxXjsVA'

In [35]:
# Считывание данных

groups_df = read_csv_file(groups_link)
groups_add_df = read_csv_file(groups_add_link)
active_studs_df = read_csv_file(active_studs_link)
checks_df = read_csv_file(checks_link)

### Работа с исходными таблицами

In [57]:
# Проверка на корректность данных: в доп файле не дублируется id пользователя

groups_add_df.id.isin(groups_df.id).sum()

0

In [67]:
# Переименовал колонки

active_studs_df = active_studs_df.rename(columns={'"student_id"': 'id'})
checks_df = checks_df.rename(columns={'"student_id"': 'id', '"rev"': 'rev'})

In [71]:
# Совместил всех пользователей участвующих в A/B-тестировании 

all_users = pd.concat([groups_df, groups_add_df], ignore_index=True)

Применяю left join для таблицы с юзерами, которые зашли на платформу в дни проведения эксперемента с таблицой, содержащей всех пользователей, участвующих в A/B-тестировании.

Далее работаю с данной таблицей.

In [83]:
active_users = active_studs_df.merge(all_users, how='left', on='id')

In [89]:
# Джойн выполнен успешно

active_users.isnull().sum()

id     0
grp    0
dtype: int64

In [97]:
# Количество строк в таблице

active_users.shape[0]

8341

In [90]:
active_users.head(5)

,id,grp
0,581585,A
1,5723133,A
2,3276743,B
3,4238589,A
4,4475369,B


In [101]:
# Количество пользователей контрольной группы

active_users[active_users.grp == 'A'].id.count()

1538

In [102]:
# Количество пользователей целевой группы

active_users[active_users.grp == 'B'].id.count()

6803

In [110]:
# Количество строк в таблице checks_df

checks_df.shape[0]

541

Количество уникальных значений id в таблице checks_df равно количеству строк.

Значит каждый пользователь, совершивший платное действие, произвел одну оплату в дни проведения тестирования.

In [111]:
# Количество уникальных значений id в таблице checks_df

checks_df.id.nunique()

541

### Работа с итоговыми таблицами

In [119]:
# Джойн таблицы с оплатами пользователей с таблицей, содержащей активных пользователей во время тестирования

purchasing_users = checks_df.merge(active_users, how='inner', on='id')

149 пользователей, совершивших оплаты в дни проведения тестирования, не участвовали в эксперименте 

In [120]:
checks_df.shape[0] - purchasing_users.shape[0]

149

In [124]:
purchasing_users[purchasing_users.grp == 'A'].id.count()

78

In [125]:
purchasing_users[purchasing_users.grp == 'B'].id.count()

314

In [122]:
active_users

,id,grp
0,581585,A
1,5723133,A
2,3276743,B
3,4238589,A
4,4475369,B
...,...,...
8336,1794344,B
8337,296883,B
8338,3598448,B
8339,3610547,B


In [121]:
purchasing_users

,id,rev,grp
0,1627,990.0000,A
1,100300,990.0000,B
2,108270,290.0000,B
3,264473,1900.0001,B
4,274352,690.0000,B
...,...,...,...
387,5645737,1900.0000,B
388,5651566,1900.0000,B
389,5662322,1900.0000,B
390,5681918,1900.0000,B
